# Design a system that detects drowsiness in a user by analyzing their eye state (open or closed) in real-time using the principles you learned from the emotion detection model.

In [ ]:
# Install dlib
!pip install dlib

# Install OpenCV
!pip install opencv-python

# Install numpy
!pip install numpy



In [ ]:
import cv2
import dlib
import numpy as np
from collections import deque

# Load the dlib face detector and facial landmark predictor
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("D:\\NullClassTask\\Task1\\shape_predictor_68_face_landmarks.dat\\shape_predictor_68_face_landmarks.dat")


# Function to detect face landmarks and extract eye regions
def detect_and_extract_eye_regions(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)
    
    if len(faces) > 0:
        face_landmarks = landmark_predictor(gray, faces[0])
        face_landmarks = np.array([[p.x, p.y] for p in face_landmarks.parts()])
        return face_landmarks
    else:
        return None

# Function to extract eye regions from the face
def extract_eye_regions(face_img, left_eye_landmarks, right_eye_landmarks):
    left_eye_region = cv2.convexHull(left_eye_landmarks)
    right_eye_region = cv2.convexHull(right_eye_landmarks)
    
    left_eye_mask = np.zeros(face_img.shape[:2], dtype=np.uint8)
    right_eye_mask = np.zeros(face_img.shape[:2], dtype=np.uint8)
    
    cv2.drawContours(left_eye_mask, [left_eye_region], -1, (255, 255, 255), thickness=cv2.FILLED)
    cv2.drawContours(right_eye_mask, [right_eye_region], -1, (255, 255, 255), thickness=cv2.FILLED)
    
    left_eye = cv2.bitwise_and(face_img, face_img, mask=left_eye_mask)
    right_eye = cv2.bitwise_and(face_img, face_img, mask=right_eye_mask)
    
    return left_eye, right_eye

# Function to calculate the eye aspect ratio (EAR)
def eye_aspect_ratio(eye_landmarks):
    # Calculate the euclidean distances between the vertical eye landmarks
    vertical_dist_1 = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    vertical_dist_2 = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
    
    # Calculate the euclidean distance between the horizontal eye landmarks
    horizontal_dist = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
    
    # Calculate the eye aspect ratio (EAR)
    ear = (vertical_dist_1 + vertical_dist_2) / (2.0 * horizontal_dist)
    return ear

# Function to detect eye state based on eye aspect ratio
def detect_eye_state(ear_left, ear_right, ear_threshold=0.3):
    avg_ear = (ear_left + ear_right) / 2.0
    if avg_ear < ear_threshold:
        return 'Closed'
    else:
        return 'Open'

# Main loop to capture video from the webcam
video_capture = cv2.VideoCapture(0)  # Use '0' for the default webcam

# Initialize deque to store EAR values for the past few frames
ear_deque = deque(maxlen=10)  # Change this value based on the desired history length

while True:
    ret, frame = video_capture.read()

    if ret:
        face_landmarks = detect_and_extract_eye_regions(frame)
        
        if face_landmarks is not None:
            left_eye_landmarks = face_landmarks[36:42]
            right_eye_landmarks = face_landmarks[42:48]

            left_eye, right_eye = extract_eye_regions(frame, left_eye_landmarks, right_eye_landmarks)
            
            left_eye_ear = eye_aspect_ratio(left_eye_landmarks)
            right_eye_ear = eye_aspect_ratio(right_eye_landmarks)
            
            # Add the average EAR to the deque
            avg_ear = (left_eye_ear + right_eye_ear) / 2.0
            ear_deque.append(avg_ear)

            # Check if the EAR values are below a threshold for a prolonged period (drowsiness detection)
            if len(ear_deque) == ear_deque.maxlen and all(ear < 0.3 for ear in ear_deque):
                cv2.putText(frame, "Drowsy", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            eye_state = detect_eye_state(left_eye_ear, right_eye_ear)

            # Draw eye state on the frame
            cv2.putText(frame, f"Eye State: {eye_state}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow("Eye State Detection", frame)

    # Exit the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
video_capture.release()
cv2.destroyAllWindows()
